In [82]:
import numpy as np
import pandas as pd

In [111]:
books = pd.read_csv('books.csv')
users = pd.read_csv('users.csv')
ratings = pd.read_csv('ratings.csv')

C:\Users\naman\AppData\Local\Temp\ipykernel_15460\3819754567.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('books.csv')


In [112]:
ratings_with_name = ratings.merge(books,on='ISBN')

In [85]:
x=ratings_with_name.groupby('User-ID').count()['Book-Rating']>50
t=x[x].index#boolean indexing, will only consist of true values

In [86]:
filtered_rating=ratings_with_name[ratings_with_name['User-ID'].isin(t)]#this too is boolean indexing

In [87]:
y=filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books=y[y].index

In [88]:
final_ratings=filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]#this too is boolean indexing

In [89]:
pt=final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [90]:
pt.fillna(0,inplace=True)

In [91]:
pt

User-ID,243,254,507,638,643,741,882,929,1211,1424,...,277928,277965,278026,278137,278144,278188,278418,278582,278633,278843
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204 Rosewood Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
similarity_scores = cosine_similarity(pt)
similarity_scores.shape

(1350, 1350)

In [94]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [95]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['American Psycho (Vintage Contemporaries)',
  'Bret Easton Ellis',
  'http://images.amazon.com/images/P/0679735771.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg']]

In [96]:
import pickle

In [97]:
pickle.dump(similarity_scores,open('similarity.pkl','wb'))

In [98]:
pickle.load(open('similarity.pkl','rb'))

array([[1.        , 0.04612255, 0.06374442, ..., 0.05526131, 0.03761358,
        0.02104474],
       [0.04612255, 1.        , 0.01695641, ..., 0.03826821, 0.12763956,
        0.09490141],
       [0.06374442, 0.01695641, 1.        , ..., 0.07303747, 0.        ,
        0.        ],
       ...,
       [0.05526131, 0.03826821, 0.07303747, ..., 1.        , 0.04144573,
        0.01093041],
       [0.03761358, 0.12763956, 0.        , ..., 0.04144573, 1.        ,
        0.06627547],
       [0.02104474, 0.09490141, 0.        , ..., 0.01093041, 0.06627547,
        1.        ]])

In [99]:
books=final_ratings.drop_duplicates(subset=['Book-Title'])[['ISBN','Book-Rating','Book-Title','Book-Author','Image-URL-M']]
books

,ISBN,Book-Rating,Book-Title,Book-Author,Image-URL-M
63,0446520802,0,The Notebook,Nicholas Sparks,http://images.amazon.com/images/P/0446520802.0...
184,038550120X,0,A Painted House,JOHN GRISHAM,http://images.amazon.com/images/P/038550120X.0...
368,0425115801,0,Lightning,Dean R. Koontz,http://images.amazon.com/images/P/0425115801.0...
502,0449006522,8,Manhattan Hunt Club,JOHN SAUL,http://images.amazon.com/images/P/0449006522.0...
613,0553561618,0,Dark Paradise,TAMI HOAG,http://images.amazon.com/images/P/0553561618.0...
...,...,...,...,...,...
524627,0425127583,8,The Mask,Dean R. Koontz,http://images.amazon.com/images/P/0425127583.0...
529982,0425077047,0,And Ladies of the Club,Helen Hooven Santmyer,http://images.amazon.com/images/P/0425077047.0...
538480,0553280945,8,The Mammoth Hunters (Earth's Children (Paperba...,JEAN M. AUEL,http://images.amazon.com/images/P/0553280945.0...
547543,0553573209,0,Firefly Beach,Luanne Rice,http://images.amazon.com/images/P/0553573209.0...


In [100]:
pickle.dump(books,open('book_list.pkl','wb'))

In [104]:
pickle.dump(pt,open('pt.pkl','wb'))

In [125]:
rating = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
rating.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
rating

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [126]:
avg_rating = ratings_with_name.groupby('Book-Title').mean(numeric_only=True)['Book-Rating'].reset_index()
avg_rating.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [151]:
popular_books = rating.merge(avg_rating_df,on='Book-Title')
popular_books = popular_books[popular_books['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)
popular_books = popular_books.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']].reset_index()
popular_books

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [157]:
pickle.dump(popular_books,open('popular_books.pkl','wb'))